In [15]:
%load_ext autoreload
%autoreload 2
from transformers import BertModel, BertConfig
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import numpy as np
import featurize_dset
from datasets import load_from_disk
import pickle as pkl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**feature extraction**

In [2]:
dataset = datasets.load_dataset('sst2')
dataset

Using custom data configuration default
Reusing dataset sst2 (/tmp/.xdg_cache_vision/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [3]:
dset_small = dataset['validation'] #[:10]
dset_small = dset_small.filter(lambda x: len(x["sentence"]) < 20)
dset_small

Loading cached processed dataset at /tmp/.xdg_cache_vision/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-7876086904868ef1.arrow


Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 8
})

# ngram feature extraction
let's map each sequence to the sum of features obtained by each token (tutorial [here](https://huggingface.co/course/chapter3/2?fw=pt)).

In [6]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BertModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
embs_ex = featurize_dset.featurize_function({'sentence': 'not very good'})

In [ ]:
# small example
featurized_dataset = dset_small.map(featurize_dset.featurize_function) #, batched=True)
embs = np.array(featurized_dataset['embs']).squeeze()
print(embs.shape)

In [11]:
featurized_dataset.save_to_disk("data/processed/featurized_example")

In [13]:
reloaded_dataset = load_from_disk("data/processed/featurized_example")

In [16]:
embs = np.array(reloaded_dataset['embs']).squeeze()
print(embs.shape)
pkl.dump(embs, open('data/processed/featurized_example.pkl', 'wb'))

(8, 768)


In [17]:
embs = pkl.load(open('data/processed/featurized_example.pkl', 'rb'))

In [ ]:
for k in reloaded_dataset:
    print(k['embs'])